# 文本分类实例

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [2]:
# 通过新方法加载数据和数据清洗
import datasets
dataset = datasets.load_dataset("csv", data_files="./ChnSentiCorp_htl_all.csv", split="train")
print(len(dataset))
dataset = dataset.filter(lambda x: x["review"] is not None)
print(len(dataset))
# 划分数据集
split_dataset = dataset.train_test_split(test_size=0.1)
split_dataset

7766
7765


DatasetDict({
    train: Dataset({
        features: ['label', 'review'],
        num_rows: 6988
    })
    test: Dataset({
        features: ['label', 'review'],
        num_rows: 777
    })
})

In [3]:
# 进行分词
tokenizer = AutoTokenizer.from_pretrained("hfl/rbt3")

def process_function(example):
    tokenized_example = tokenizer(example["review"], max_length=128, truncation=True)
    tokenized_example["labels"] = example["label"]
    return tokenized_example

tokenized_datasets = split_dataset.map(function=process_function, batched=True, remove_columns=split_dataset["train"].column_names)

tokenized_datasets

Map:   0%|          | 0/6988 [00:00<?, ? examples/s]

Map:   0%|          | 0/777 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 6988
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 777
    })
})

In [6]:
# 创建dataset
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding

train_loader = DataLoader(tokenized_datasets["train"], batch_size=32, shuffle=True, collate_fn=DataCollatorWithPadding(tokenizer))

valid_loader = DataLoader(tokenized_datasets["test"], batch_size=64, shuffle=False, collate_fn=DataCollatorWithPadding(tokenizer))

In [7]:
next(enumerate(train_loader))[1]

{'input_ids': tensor([[ 101, 6821,  702,  ...,    0,    0,    0],
        [ 101,  868,  711,  ...,    0,    0,    0],
        [ 101, 3302, 1218,  ...,    0,    0,    0],
        ...,
        [ 101, 4692, 6814,  ...,  817, 2347,  102],
        [ 101, 1343, 5722,  ..., 2791, 7313,  102],
        [ 101, 2137, 4638,  ..., 6574, 7030,  102]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]]), 'labels': tensor([1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
        0, 0, 1, 1, 1, 1, 1, 1])}

In [8]:
from torch.optim import Adam
import torch

# 模型及优化器定义
model = AutoModelForSequenceClassification.from_pretrained("hfl/rbt3")
optimizer = Adam(model.parameters(), lr=0.00002)

if torch.cuda.is_available():
    model.cuda()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/rbt3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
# 模型训练
def train(epochs, log_step=10):
    for epoch in range(epochs):
        step=0
        model.train()
        print(f"<<<<<<<<Training on epoch{epoch} >>>>>>>>")
        for batch in train_loader:
            if torch.cuda.is_available():
                batch = {k: v.cuda() for k, v in batch.items()}
            optimizer.zero_grad()
            output = model(**batch)
            output.loss.backward()
            optimizer.step()

            if step % log_step == 0:
                print(f"global_step: {step}, loss: {output.loss.item()}")
            step += 1
        evaluate()


# 模型验证
def evaluate():
    model.eval()
    acc_num = 0
    total = 0
    # 该方法比no_grad()更加的优化了测试的内存管理方面
    with torch.inference_mode():
        for batch in valid_loader:
            if torch.cuda.is_available():
                batch = {k: v.cuda() for k, v in batch.items()}
                output = model(**batch)
                pred = torch.argmax(output.logits, dim=-1)
                for p, l in zip(pred, batch["labels"]):
                    total += 1
                    if p == l:
                        acc_num += 1
        print(f"acc: {acc_num / total}")


In [10]:
train(epochs=5)

<<<<<<<<Training on epoch0 >>>>>>>>


C:\Users\31825\.conda\envs\transformers\lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


global_step: 0, loss: 0.6127764582633972
global_step: 10, loss: 0.7215803265571594
global_step: 20, loss: 0.488822340965271
global_step: 30, loss: 0.5125473141670227
global_step: 40, loss: 0.5366478562355042
global_step: 50, loss: 0.39659732580184937
global_step: 60, loss: 0.3305644094944
global_step: 70, loss: 0.38386204838752747
global_step: 80, loss: 0.19288167357444763
global_step: 90, loss: 0.33696407079696655
global_step: 100, loss: 0.2079067975282669
global_step: 110, loss: 0.257258802652359
global_step: 120, loss: 0.2732982933521271
global_step: 130, loss: 0.25069162249565125
global_step: 140, loss: 0.4484230577945709
global_step: 150, loss: 0.14048022031784058
global_step: 160, loss: 0.2136348932981491
global_step: 170, loss: 0.22057226300239563
global_step: 180, loss: 0.4068312644958496
global_step: 190, loss: 0.15010777115821838
global_step: 200, loss: 0.17918621003627777
global_step: 210, loss: 0.3158644735813141
acc: 0.8764478764478765
<<<<<<<<Training on epoch1 >>>>>>>>
g

KeyboardInterrupt: 

In [18]:
# 预测
test = "这家酒店不错，饭很好吃"
with torch.inference_mode():
    test_tensor = tokenizer(test, return_tensors="pt")
    test_tensor = {k: v.cuda() for k, v, in test_tensor.items()}
    test_result = model(**test_tensor).logits
    pred = torch.argmax(test_result, dim=-1)
    print(pred.item())

1


In [19]:
# 创建pipeline
from transformers import pipeline

pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device="cuda:0")

Device set to use cuda:0


In [20]:
pipe("这家酒店不错，饭很好吃")

[{'label': 'LABEL_1', 'score': 0.9989665746688843}]